In [8]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, FloatType


In [9]:
def write_DB (parquet_DF, table):
    try:
        parquet_DF.write.jdbc(
            url=jdbc_url,
            table=table,
            mode="overwrite",
            properties=properties
        )
    except Exception as error:
        logger.error("Error ruta .yaml:" + str(error))

In [10]:

spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .config("spark.jars", "/home/illidan/proyecto_desde0/postgre/jars/postgresql-42.7.4.jar") \
    .getOrCreate()
)



spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))

errores_detectados = []


14:05:22.068 [Thread-4] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [11]:
#BD conection

try:
    jdbc_url = "jdbc:postgresql://localhost:5433/sparkdb_dev"
    properties = {
        "user": config["sparkdb_dev"]["user"],
        "password": config["sparkdb_dev"]["password"],
        "driver": "org.postgresql.Driver"
    }
except Exception as error:
    logger.error("Error conection BD:" + str(error))



In [12]:
#Read parquet file
try:
    read_avg_flight = config['Parquet_file']['avg_flight']
    read_airline_in_airport = config['Parquet_file']['airline_in_airport']
    read_flights_per_cancell = config['Parquet_file']['flights_per_cancell']
    read_airport_notin_thelist = config['Parquet_file']['airport_notin_thelist']
except Exception as error:
    logger.error("Error read parquet:" + str(error))


try:
    avg_flight = spark.read.parquet(read_avg_flight)
    airline_in_airport =  spark.read.parquet(read_airline_in_airport)
    flights_per_cancell = spark.read.parquet(read_flights_per_cancell)
    airport_notin_thelist = spark.read.parquet(read_airport_notin_thelist)
except Exception as error:
    logger.error("Error read parquet:" + str(error))

14:05:22.110 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.InMemoryFileIndex - It took 2 ms to list leaf files for 1 paths.
14:05:22.159 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_1_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.3 MiB)
14:05:22.163 [Thread-4] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
14:05:22.165 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 4 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
14:05:22.165 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 4 (parquet at NativeMethodAccessorImpl.java:0)
14:05:22.165 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_2_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.4 MiB)
14:05:22.166 [dag-scheduler-event-loop]

In [13]:
#write parquet file into DB

write_DB(avg_flight, "avg_flight")
write_DB(airline_in_airport,"airline_in_airport")
write_DB(flights_per_cancell,"flights_cancellation")
write_DB(airport_notin_thelist,"airport_notin_thelist")




14:05:22.944 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_7_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.3 MiB)
14:05:22.951 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_5_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.3 MiB)
14:05:22.957 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_6_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.4 MiB)
14:05:22.962 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_4_piece0 on 172.23.57.81:34293 in memory (size: 37.2 KiB, free: 434.4 MiB)
14:05:23.616 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: 
14:05:23.617 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: 
14:05:23.955 [Thread-4]

14:05:25.440 [Executor task launch worker for task 0.0 in stage 8.0 (TID 8)] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 21.70337 ms
14:05:25.444 [Executor task launch worker for task 0.0 in stage 8.0 (TID 8)] INFO  org.apache.spark.sql.execution.datasources.FileScanRDD - Reading File path: file:///home/illidan/proyecto_desde0/archivos_parquet/avg_flight/part-00000-993f1908-08c5-4c37-91c4-d7ad5e3d16a3-c000.snappy.parquet, range: 0-1142, partition values: [empty row]
14:05:25.569 [Executor task launch worker for task 0.0 in stage 8.0 (TID 8)] INFO  org.apache.hadoop.io.compress.CodecPool - Got brand-new decompressor [.snappy]
14:05:25.770 [Executor task launch worker for task 0.0 in stage 8.0 (TID 8)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 8.0 (TID 8). 1800 bytes result sent to driver
14:05:25.771 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 8.0 (TID 8) in 

14:05:25.971 [dag-scheduler-event-loop] INFO  org.apache.spark.SparkContext - Created broadcast 11 from broadcast at DAGScheduler.scala:1585
14:05:25.971 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting 1 missing tasks from ResultStage 9 (MapPartitionsRDD[27] at jdbc at NativeMethodAccessorImpl.java:0) (first 15 tasks are for partitions Vector(0))
14:05:25.972 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Adding task set 9.0 with 1 tasks resource profile 0
14:05:25.974 [dispatcher-event-loop-2] INFO  org.apache.spark.scheduler.TaskSetManager - Starting task 0.0 in stage 9.0 (TID 9) (172.23.57.81, executor driver, partition 0, PROCESS_LOCAL, 9882 bytes) 
14:05:25.975 [Executor task launch worker for task 0.0 in stage 9.0 (TID 9)] INFO  org.apache.spark.executor.Executor - Running task 0.0 in stage 9.0 (TID 9)
14:05:26.015 [Executor task launch worker for task 0.0 in stage 9.0 (TID 9)] INFO  org.apache.spark.sql.catal